In [1]:
import httpx
import os
from typing import Dict, Any
from fastapi import APIRouter, Depends, HTTPException, Query
from sqlalchemy.orm import Session
from sqlalchemy.exc import IntegrityError
from sqlalchemy import and_, or_
from typing import Optional

In [2]:
URGENCY_API_BASE = os.getenv("URGENCY_API_BASE", "https://kar137-sambodhan-urgency-classifier-space.hf.space")
DEPARTMENT_API_BASE = os.getenv("DEPARTMENT_API_BASE", "https://mr-kush-sambodhan-department-classifier.hf.space")
CLASSIFIER_TIMEOUT_SECONDS = int(os.getenv("CLASSIFIER_TIMEOUT_SECONDS", "30"))


In [3]:
async def predict_urgency(text: str) -> Dict[str, Any]:
    """
    Call external urgency classifier API.
    Returns: {"urgency": int, "confidence": float}
    """
    try:
        async with httpx.AsyncClient(timeout=CLASSIFIER_TIMEOUT_SECONDS) as client:
            response = await client.post(
                f"{URGENCY_API_BASE}/predict_urgency",
                json={"text": text}
            )
            response.raise_for_status()
            result = response.json()
            # Map model label to int code
            label_map = {
                "NORMAL": 0,
                "URGENT": 1,
                "HIGHLY URGENT": 2
            }
            urgency_code = label_map.get(result.get("label", "").upper(), 0)
            return {
                "urgency": urgency_code,
                "confidence": result.get("confidence", 0.0),
                "label": result.get("label", "")
            }
    except Exception as e:
        print(f"Urgency classifier error: {str(e)}")
        # Fallback to default urgency if classifier fails
        return {"urgency": 0, "confidence": 0.0}



In [4]:

async def predict_department(text: str) -> Dict[str, Any]:
    """
    Call external department classifier API.
    Returns: {"department": int, "confidence": float}
    """
    try:
        async with httpx.AsyncClient(timeout=CLASSIFIER_TIMEOUT_SECONDS) as client:
            response = await client.post(
                f"{DEPARTMENT_API_BASE}",
                json={"text": text, "return_probabilities": False}
            )
            response.raise_for_status()
            result = response.json()
            
            # Map string department to int code
            dept_map = {
                "infrastructure": 1,
                "health": 2,
                "education": 3,
                "environment": 4
            }
            dept_code = dept_map.get(result.get("department", "").lower(), 0)
            
            return {
                "department": dept_code,
                "confidence": result.get("confidence", 0.0)
            }
    except Exception as e:
        print(f"Department classifier error: {str(e)}")
        # Fallback to unclassified if classifier fails
        return {"department": 0, "confidence": 0.0}



In [5]:
text = "there is no electricity. this is very urgent"  # Example grievance text

In [6]:
department_result = await predict_department(text)
urgency_result = await predict_urgency(text)
print("Department Prediction:", department_result)
print("Urgency Prediction:", urgency_result)

Department classifier error: Client error '405 Method Not Allowed' for url 'https://mr-kush-sambodhan-department-classifier.hf.space'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/405
Department Prediction: {'department': 0, 'confidence': 0.0}
Urgency Prediction: {'urgency': 2, 'confidence': 0.9883, 'label': 'HIGHLY URGENT'}
